<a href="https://colab.research.google.com/github/alanntl/SELGO-LITE/blob/alanntl/SLEGO_LITE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SLEGO Project: UNSW CSE PhD Research - Alan Siu Lung Ng
https://github.com/alanntl/SELGO-LITE

# Connect this Notebook to your GoogleDrive
paste this code on terminal if you wanna run colab using local runtime:

  jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

For running SLEGO locally, you need to download Google Drive on your local computer to make the local path available.

IF you are collaborating data analytics using slego, please share your slego folder to your teammate, and the one being shared need to make this SLEGO folder a "shortcut" to google drive's main folder.

# Setup the app foldersystem for slego

In [7]:
# environment_setup.py

import os
import sys
import subprocess
from typing import Dict, Any, Optional

def detect_environment() -> str:
    """Detect the current runtime environment."""
    if 'google.colab' in sys.modules:
        return 'colab'
    elif 'CODESPACES' in os.environ and os.environ['CODESPACES'] == 'true':
        return 'github-codespaces'
    else:
        return 'local-jupyter'

def get_environment_config() -> Dict[str, Any]:
    """Get the configuration based on the current runtime environment."""
    config = {}
    env = detect_environment()
    
    if env == 'colab':
        print("Running in Google Colab environment.")
        config['drive_mainfolder'] = '/content/drive/MyDrive/SLEGO'
        config['drive_folder'] = '/content/drive/MyDrive/'
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
    elif env == 'github-codespaces':
        print("Running in GitHub Codespaces environment.")
        config['drive_mainfolder'] = '/workspaces/SELGO-LITE'
        config['drive_folder'] = '/workspaces/'
    else:  # local-jupyter
        print("Running in a local Jupyter environment.")
        gmailaccount = os.environ.get('GMAIL_ACCOUNT', 'default@gmail.com')
        config['drive_mainfolder'] = f"/Users/an/Library/CloudStorage/GoogleDrive-{gmailaccount}/My Drive/SLEGO"
        config['drive_folder'] = f"/Users/an/Library/CloudStorage/GoogleDrive-{gmailaccount}/My Drive/"
    
    config['repo_url'] = 'https://github.com/alanntl/SELGO-LITE.git'
    config['slego_env'] = f"{config['drive_folder']}/slego_env_v0_0_1"
    config['requirements_file'] = f"{config['drive_mainfolder']}/requirements.txt"
    
    # Set up workspace folders
    config['folder_path'] = f"{config['drive_mainfolder']}/slegospace"
    config['dataspace'] = f"{config['folder_path']}/dataspace"
    config['recordspace'] = f"{config['folder_path']}/recordspace"
    config['functionspace'] = f"{config['folder_path']}/functionspace"
    config['knowledgespace'] = f"{config['folder_path']}/knowledgespace"
    
    os.environ['DRIVE_MAINFOLDER'] = config['drive_mainfolder']
    os.environ['DRIVE_FOLDER'] = config['drive_folder']
    
    return config

def run_command(command: list, check: bool = True, **kwargs) -> Optional[bool]:
    """Run a subprocess command with error handling."""
    try:
        subprocess.run(command, check=check, **kwargs)
    except subprocess.CalledProcessError as e:
        print(f"Error running command {' '.join(command)}: {e}")
        return None
    return True

def setup_repository(config: Dict[str, Any]):
    """Set up or update the repository based on the current environment."""
    repo_path = config['drive_mainfolder']
    repo_url = config['repo_url']

    if not os.path.exists(repo_path):
        if run_command(['git', 'clone', repo_url, repo_path]):
            print("Repository cloned.")
    else:
        os.chdir(repo_path)
        if run_command(['git', 'fetch']):
            result = subprocess.run(['git', 'diff', '--name-only', 'HEAD', 'origin/master'], capture_output=True, text=True)
            changed_files = result.stdout.splitlines()
            directory_changes = any(os.path.dirname(f) for f in changed_files)
            if directory_changes:
                if run_command(['git', 'pull']):
                    print("Repository updated with new directory changes.")
            else:
                print("No directory changes detected; no update necessary.")

def setup_virtual_environment(config: Dict[str, Any]):
    """Set up the virtual environment and install requirements."""
    env = detect_environment()
    slego_env = config['slego_env']
    requirements_file = config['requirements_file']

    if not os.path.exists(slego_env):
        run_command([sys.executable, '-m', 'pip', 'install', 'virtualenv'])
        run_command([sys.executable, '-m', 'virtualenv', slego_env])
    
    if env == 'colab':
        activate_this = f"{slego_env}/bin/activate_this.py"
        exec(open(activate_this).read(), {'__file__': activate_this})
        sys.path.append(f"{slego_env}/lib/python3.10/site-packages")
    elif env == 'github-codespaces':
        os.environ['VIRTUAL_ENV'] = slego_env
        os.environ['PATH'] = f"{slego_env}/bin:{os.environ['PATH']}"
    else:  # local-jupyter
        activate_this = f"{slego_env}/bin/activate_this.py"
        exec(open(activate_this).read(), {'__file__': activate_this})

    if os.path.exists(requirements_file):
        run_command([sys.executable, '-m', 'pip', 'install', '-r', requirements_file])

def setup_workspace(config: Dict[str, Any]):
    """Set up the workspace folders and change the working directory."""
    for folder in [config['folder_path'], config['dataspace'], config['recordspace'], 
                   config['functionspace'], config['knowledgespace']]:
        os.makedirs(folder, exist_ok=True)
    
    os.chdir(config['folder_path'])
    print(f"Working directory changed to: {os.getcwd()}")

    if detect_environment() == 'colab':
        from google.colab import files
        files.view(config['folder_path'])

def setup_environment():
    """Set up the environment, repository, virtual environment, and workspace."""
    config = get_environment_config()
    setup_repository(config)
    setup_virtual_environment(config)
    setup_workspace(config)
    return config

# Global variable to store the configuration
global_config = None

# Automatically run setup when the module is imported
global_config = setup_environment()

Running in GitHub Codespaces environment.
No directory changes detected; no update necessary.
Working directory changed to: /workspaces/SELGO-LITE/slegospace


# Import Libraries
Import all the related libraries

In [8]:
%%time
import panel as pn
import inspect
import ast  # For safely evaluating the input string
import re
import importlib
import json
import io
import time
import param
import json
from datetime import datetime
import itertools
import pandas as pd
pn.extension()
pn.extension(sizing_mode = 'stretch_both')
pn.extension('ace', 'jsoneditor')
pn.extension('tabulator')


CPU times: user 82.1 ms, sys: 4.83 ms, total: 87 ms
Wall time: 209 ms


# Select which modules to import

In [9]:
%%time

folder_path = global_config['folder_path']
def delete_func_file(func_file_path):
    # Check if the file exists
    if os.path.exists(func_file_path):
        # Delete the file
        os.remove(func_file_path)
        #print(f"File {func_file_path} has been deleted.")
    else:
        print(f"No file named {func_file_path} found.")


functionspace ='/functionspace'
# Step 3: Get the list of .py files in the folder
py_files = [file for file in os.listdir(folder_path+functionspace) if file.endswith('.py')]

# New Step: Check if func.py exists and delete it
func_file_path = 'func.py'
delete_func_file(func_file_path)

funcfilecombo = pn.widgets.MultiChoice(name='Select Function',
                                       value=['util.py',
                                              'func_data_preprocss.py',
                                              'func_moving_avg_plot.py',
                                              'llm.py',
                                              'func_viz.py',
                                              'func_eda.py',
                                              'func_uci_dataset.py',
                                              'webscrape.py',
                                              'func_arxiv.py',
                                              'func_backtest.py',
                                              'func_autogluon.py'],
                                       options=py_files, height=200)

# create funcfilecombo_change function
def funcfilecombo_change(event):
    delete_func_file(func_file_path)

    py_files = funcfilecombo.value
    # Step 4 (Modified): Create a new file named func.py in the current repository
    with open(func_file_path, 'w') as func_file:
        # Step 5: Iterate over each .py file and append its content to func.py
        for py_file in py_files:
            print(py_file)
            file_path = os.path.join(folder_path+functionspace, py_file)
            with open(file_path, 'r') as file:
                func_file.write(file.read() + '\n')
    import func
    importlib.reload(func)


# param watch
funcfilecombo.param.watch(funcfilecombo_change, 'value')

display(funcfilecombo)

# call the event
funcfilecombo_change(None)
# !ls

BokehModel(combine_events=True, render_bundle={'docs_json': {'fe90bd9e-58c1-41e4-8099-c1b9a8e7bd95': {'version…

No file named func.py found.
util.py
func_data_preprocss.py
func_moving_avg_plot.py
llm.py
func_viz.py
func_eda.py
func_uci_dataset.py
webscrape.py
func_arxiv.py
func_backtest.py
func_autogluon.py
CPU times: user 26.3 ms, sys: 4.47 ms, total: 30.8 ms
Wall time: 35.8 ms


# SLEGO APP

In [10]:
%%time
import func
importlib.reload(func)

funccombo = func._create_multi_select_combobox(func)

compute_btn = pn.widgets.Button(name='Compute', height =50,  button_type='primary')
savepipe_btn = pn.widgets.Button(name='Save Pipeline', height =35)
pipeline_text= pn.widgets.TextInput(value='', placeholder='Input Pipeline Name', height=35)
json_toggle = pn.widgets.Toggle(name='Input mode: text or form', height =35, button_type='warning')

pipeline_dict = {}
json_editor = pn.widgets.JSONEditor(value=pipeline_dict, height=400, mode='form')
input_text = pn.widgets.TextAreaInput(value='', placeholder='input the parameters', height=400)
progress_text = pn.widgets.TextAreaInput(value='',placeholder='Input your analytics query here', name='Progress msg and query inputs for recommendation:',height=150)
output_text= pn.widgets.TextAreaInput(value='',placeholder= 'Results will be shown here', name='System output message:')

recommendation_btn = pn.widgets.Button(name='Get Recommendation', height =35, button_type='success')
recomAPI_text= pn.widgets.TextInput(value='', placeholder='Your AI API key', height=35)


file_text = pn.widgets.TextInput(value='/dataspace', placeholder='Input the file name')
folder_select= pn.widgets.Select(name= 'Select Folder', options=[item for item in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, item))] +['/'], value= 'dataspace', height=50)

filefolder_confirm_btn = pn.widgets.Button(name='Confirm')

file_view = pn.widgets.Button(name='View')
file_download = pn.widgets.Button(name='Download')
file_upload = pn.widgets.Button(name='Upload')
file_input = pn.widgets.FileInput( name='Upload file')
file_delete = pn.widgets.Button(name='Delete')

selected_folder_path = folder_path + str(file_text.value)
file_list= os.listdir(selected_folder_path)
df_file = pd.DataFrame(file_list, columns=['Filter Files :'])
file_table = pn.widgets.Tabulator(df_file, theme = 'semantic-ui', header_filters=True,layout='fit_data_table', show_index=False,margin=(0,0,30,0))


def folder_select_changed(event):
    file_text.value = '/'+ str(folder_select.value)
    on_filefolder_confirm_btn_click(None)


def json_editor_change(event):

    text = str(json_editor.value)
    text = re.sub(r'\bfalse\b', 'False', text, flags=re.IGNORECASE)
    text = text.replace("'", '"')

    input_text.value = text


# recommendation_btn param watch
def recommendation_btn_clicked(event):
    if event.new:
        output_text.value = 'recommenda clicked, getting recommendation from the AI...'
        ui_input = str(json_editor.value)

        prompt_format ='''
                            {
                              "microservice1": {
                                  "param1": "default value 1",
                                  "param2":"default value 2",
                                  "param3":"default value 3",
                                  },

                              "microservice2": {
                                  "param1": "default value 1",
                                  "param2":"default value 2",
                                },
                              }
                             '''
        instruction = f"generate the pipeline dictionary for me base on my query and input. the format shoud be similar to {prompt_format}, before you give the answer, please check if user input anything in the UI textbox, if they did, please recommend a piepline related to the input:{ui_input} "
        query = progress_text.value

        response = func.chatgpt_chat(model='gpt-4-turbo',
                                      user_input_file='knowledgespace/knowledge.json',
                                      output_text_file='dataspace/gpt_output_text.txt',
                                      output_json_file='dataspace/gpt_output_full.json',
                                      temperature=1,
                                      max_tokens=4096,
                                      top_p=1,
                                      frequency_penalty=0,
                                      presence_penalty=0,
                                      api_key= recomAPI_text.value,
                                      user_message= instruction + query)
        output_text.value = response


def json_toggle_clicked(event):
    if event.new:
        widget_tab.active=1
    else:
        widget_tab.active=0

#
def get_doc_string(pipeline):
    text = input_text.value
    output=''
    #data = ast.literal_eval(text)
    data = json.loads(text)

    data.keys()
    # loop keys
    for key in data.keys():
        output +='#######'+str(key)+'#######\n'
        try:
            output += eval(f'func.{key}.__doc__')+'\n'
        except:
            output += 'No docstring found for this function\n'
    return output

def input_text_change(event):
    pipeline_dict={}
    text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
    text = text.replace("'", '"')

    try:
        #pipeline_dict = ast.literal_eval(text)
        pipeline_dict = json.loads(text)
        pipeline_dict_json = json.dumps(pipeline_dict, indent=4)

        input_text.value = pipeline_dict_json
        json_editor.value = json.loads(pipeline_dict_json)

        output_text.value += '\n Input changed!'

    except ValueError as e:
        output_text.value += f'\n Error parsing input: {e}'


def funccombo_change(event):
    output_text.value = ''
    list_funcs = funccombo.value
    list_params =[]

    for funcchoice in funccombo.value:
      function=  eval('func.'+funcchoice)
      list_params.append(func._extract_parameter(function))

    funcs_params = dict(zip(list_funcs,list_params))
    formatted_data = json.dumps(funcs_params, indent=5)

    json_editor.value = funcs_params

    input_text.value = str(formatted_data)
    output_text.value = get_doc_string(input_text.value)
    progress_text.value = f'selected {funccombo.value}!'



def save_record(space, data, pipeline_name=None):
    """
    Saves the given data structure as a JSON file named with the current datetime.

    Parameters:
    - data: The data structure to be saved as JSON.
    """
    if pipeline_name is None:
      # Generate a filename based on the current datetime, e.g., "record_20230405_153000.json"
      filename = datetime.now().strftime("record_%Y%m%d_%H%M%S.json")
    else:
      filename = pipeline_name+'.json'

    # Define the full path where you want to save the file, here assuming current directory
    full_path = './'+space+'/' + filename

    # Serialize and save the data structure to a file
    with open(full_path, "w") as file:
        json.dump(data, file, indent=5)




def compute_btn_clicked(event):
    progress_text.value = 'Computing...'

    if widget_tab.active==0:
        widget_tab.active=1
        widget_tab.active=0
    else:
        widget_tab.active=0
        widget_tab.active=1
    time.sleep(1)
    # widget_tab.active=0

    pipeline_dict = json_editor.value
    output_text.value = ''

    for function_name, parameters in pipeline_dict.items():
        progress_text.value = f'Computing {function_name}...'

        try:
            module = 'func.'
            start_time = time.time()  # Start the timer

            # Dynamically execute the function
            function = eval(module + function_name)
            result = function(**parameters)
            # Convert result to string
            result_string = str(result)
            # Create an iterator for words
            words_iterator = iter(result_string.split())
            # Use itertools.islice to get the first 1000 words without creating a full list
            first_x_words = itertools.islice(words_iterator, 500)
            # Join the words and append to output_text.value
            # Computation time
            compute_time = time.time() - start_time

            output_text.value += f"\n===================={function_name}====================\n\n"
            output_text.value += f"Function computation Time: {compute_time:.4f} seconds\n"
            output_text.value += f"\n"
            output_text.value += " ".join(first_x_words)

        except Exception as e:
            # Handle errors by appending them to the output text
            output_text.value += f"\n===================={function_name}====================\n\n"
            output_text.value += f"Error occurred: {str(e)}\n"

    save_record('recordspace', pipeline_dict)
    progress_text.value = 'Done!'
    on_filefolder_confirm_btn_click(None)

def save_pipeline(event):
    if pipeline_text.value == '':
      pipeline_name = '__'
    else:
      pipeline_name = pipeline_text.value
    text = input_text.value
    text = re.sub(r'\bfalse\b', 'False', input_text.value, flags=re.IGNORECASE)
    data = ast.literal_eval(text)
    save_record('knowledgespace', data, pipeline_name)
    on_filefolder_confirm_btn_click(None)

def on_file_buttons_click(event):
    output_text.value=''
    file_lsit =file_table.selected_dataframe.values.tolist()

    if len(file_lsit) != 0:
        if event.obj.name == 'View':
            output_text.value = ''
            for filename in file_lsit:
                output_text.value +=f"\n\n===================={str(filename)}====================\n\n"
                file_path = folder_path + str(file_text.value) + '/' + filename[0]
                with open(file_path, 'r') as file:
                    output_text.value += file.read()
        elif event.obj.name == 'Download':
            output_text.value = 'The file is already saved to your slegospace folder!'
        elif event.obj.name == 'Upload':
            output_text.value = 'Please put it into the slegospace folder!'
        elif event.obj.name == 'Delete':
            output_text.value = 'Please delete it from your slegospace folder!'
    else:
        output_text.value = 'Please select a file to view, download, upload or delete!'

def on_filefolder_confirm_btn_click(event):
    selected_folder_path = folder_path + str(file_text.value)
    file_list= os.listdir(selected_folder_path)
    df_file = pd.DataFrame(file_list, columns=['Filter Files :'])
    file_table.value = df_file

widget_tab = pn.Tabs(('json input',json_editor),('text input',input_text))

widget_input =pn.Column(pn.layout.Divider(height=10,margin=(5)),widget_tab)
widget_btns = pn.Row(savepipe_btn,pipeline_text,json_toggle )
widget_updownload = pn.Column(pn.Row(file_view, file_download, ),file_input,pn.Row(file_upload, file_delete), height=150)
widget_files = pn.Column(folder_select,pn.Row( file_text,filefolder_confirm_btn, height=55), file_table, widget_updownload, width=250, margin=(0,20,0,0))
widget_funcsel= pn.Column(funccombo, compute_btn,widget_btns )
widget_recom = pn.Row(recommendation_btn,recomAPI_text)

funccombo.param.watch(funccombo_change, 'value')
input_text.param.watch(input_text_change, 'value')
json_toggle.param.watch(json_toggle_clicked, 'value')
json_editor.param.watch(json_editor_change, 'value')
recommendation_btn.param.watch(recommendation_btn_clicked, 'value')
compute_btn.on_click(compute_btn_clicked)
savepipe_btn.on_click(save_pipeline)
filefolder_confirm_btn.on_click(on_filefolder_confirm_btn_click)
file_view.on_click(on_file_buttons_click)
file_download.on_click(on_file_buttons_click)
file_upload.on_click(on_file_buttons_click)
file_delete.on_click(on_file_buttons_click)
folder_select.param.watch(folder_select_changed, 'value')


app = pn.Row(widget_files, pn.Column(widget_funcsel,widget_input), pn.Column(widget_recom, progress_text, pn.layout.Divider(height=10,margin=(10)),output_text))


CPU times: user 57.9 ms, sys: 0 ns, total: 57.9 ms
Wall time: 65 ms


# Display the App

In [11]:
import warnings
import IPython

warnings.filterwarnings('ignore')

# Define the function to check if the runtime is Colab
def is_colab_runtime():
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Your existing code
if is_colab_runtime() == False:
    import panel as pn

    # Instantiate the template with widgets displayed in the sidebar
    template = pn.template.MaterialTemplate(
        title='SLEGO - Software Lego: A Collaborative and Modular Architecture for Data Analytics',
        sidebar=[],
    )
    # Append a layout to the main area, to demonstrate the list-like API
    template.main.append(app)
    template.show()
else:
    display(app)

Launching server at http://localhost:39437


In [4]:
# from google.colab import userdata
# userdata.get('chatgpt')